In [151]:
import xlsxwriter
import pandas as pd
import math
import time
import shutil
import xlrd

from tqdm import tqdm
from openpyxl import load_workbook
from win32com.client import DispatchEx, Dispatch
from win32api import LoadLibrary
from IPython.display import HTML, Javascript, display

PATH = "C:/Users/bloomberg3/Desktop/capital_iq/"

In [152]:
def convert_xl_serial(xl_date): 
    xl_date = int(xl_date)
    datetime_date = xlrd.xldate_as_datetime(xl_date, 0)
    date_object = datetime_date.date()
    string_date = date_object.isoformat()
    pd_datetime = pd.to_datetime(string_date)
    return pd_datetime

In [153]:
def clean_up_df(df):
    #REMOVE ROWS CORRESPONDING TO HEADERS
    df = df[~df['ticker'].str.contains('ticker')]
    
    #REMOVE ROWS THAT ARE ALL NAN (CORRESPONDING TO THERE BEING TOO LITTLE NUMBER OF HOLDERS RELATIVE TO 4000, THE NUMBER OF FUNDS CHOSEN BY ME)
    df = df.dropna(subset=['name', 'holdings', 'pct', 'ciq', 'holder_type', 'date'])
    
    #ENSURE ALL VARIABLE TYPES ARE CORRECT
    df['holdings'] = df['holdings'].astype(float)
    df['pct'] = df['pct'].astype(float)
    df['name'] = df['name'].astype(str)
    df['ciq'] = df['ciq'].astype(str)
    df['holder_type'] = df['holder_type'].astype(str)
    
    df['date'] = df['date'].astype(str)
    df['date'] = df['date'].apply(lambda x: pd.to_datetime(x) if '/' in x else convert_xl_serial(x))
    df['asof_date'] = pd.to_datetime(df['asof_date'])
    
    #REMOVE FUNDS WITH NO HOLDINGS
    df = df[df['holdings'] > 10**(-2)]
    
    return df

In [154]:
nk_df = pd.read_csv(PATH+'nk_df.csv')
ticker_list = list(nk_df['ticker'])

In [160]:
df_list = []
for ticker in tqdm(ticker_list[:]):
    p = PATH+'excel_templates/' + ticker[4:] + '.xlsx'
    df = pd.read_excel(p)
    df = clean_up_df(df)
    df_list.append(df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 231/231 [14:51<00:00,  3.86s/it]


In [161]:
full_df = pd.concat(df_list)

In [163]:
full_df.to_csv(PATH+'ownership_cleaned_r1.csv')

In [164]:
full_df.to_pickle(PATH+'ownership_cleaned_r1.pkl')